# Data cleaning and preperation

In [165]:
import pandas as pd
import re
pd.set_option('display.max_columns', None) # display all columns in a wide DataFrame
player_df = pd.read_csv('player_stats.csv')

In [166]:
player_df.head()

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,0,1,Alaa Abdelnaby,PF,23,POR,71,1,934,178,361,.493,0,0,NaN,178,361,.493,.493,76,101,.752,81,179,260,30,25,16,66,132,432,1992
1,1,2,Mahmoud Abdul-Rauf,PG,22,DEN,81,11,1538,356,845,.421,31,94,.330,325,751,.433,.440,94,108,.870,22,92,114,192,44,4,117,130,837,1992
2,2,3,Mark Acres,C,29,ORL,68,6,926,78,151,.517,1,3,.333,77,148,.520,.520,51,67,.761,97,155,252,22,25,15,33,140,208,1992
3,3,4,Michael Adams,PG,29,WSB,78,78,2795,485,1233,.393,125,386,.324,360,847,.425,.444,313,360,.869,58,252,310,594,145,9,212,162,1408,1992
4,4,5,Rafael Addison,SF,27,NJN,76,8,1175,187,432,.433,14,49,.286,173,383,.452,.449,56,76,.737,65,100,165,68,28,28,46,109,444,1992


In [167]:
player_df.shape

(19162, 32)

In [168]:
player_df.drop(columns=['Unnamed: 0', 'Rk'], inplace=True)

In [169]:
player_df = player_df[player_df['Player'] != 'Player'] # dropping extra header rows

In [170]:
player_df['Player'] = player_df['Player'].str.replace('*', '')

# players that were traded mid season have multiple rows along with a total row for that year.
player_df.drop_duplicates(subset=['Player', 'Year'], keep='first', inplace=True) # keeping only the total season row.

In [171]:
player_df.shape

(14855, 30)

In [172]:
player_df.isnull().sum() # all null values are coming from % columns - FG%, 3P%, 2P%, eFG%, FT%

Player       0
Pos          0
Age          0
Tm           0
G            0
GS           0
MP           0
FG           0
FGA          0
FG%         60
3P           0
3PA          0
3P%       2044
2P           0
2PA          0
2P%        104
eFG%        60
FT           0
FTA          0
FT%        539
ORB          0
DRB          0
TRB          0
AST          0
STL          0
BLK          0
TOV          0
PF           0
PTS          0
Year         0
dtype: int64

In [173]:
player_df[player_df['FG%'].isnull()].sample(5)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
17997,DeJon Jarreau,SG,24,IND,1,0,1,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,NaN,0,0,0,0,0,0,0,0,0,2022
14423,Danuel House Jr.,SG,23,WAS,1,0,1,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,NaN,0,1,1,0,0,0,0,0,0,2017
16274,Marques Bolden,C,21,CLE,1,0,3,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,NaN,0,2,2,0,1,0,0,1,0,2020
18082,Scottie Lewis,SG,21,CHO,2,0,7,0,0,NaN,0,0,NaN,0,0,NaN,NaN,1,2,.500,0,0,0,1,1,0,0,0,1,2022
4406,Randell Jackson,PF,24,DAL,1,0,1,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,NaN,0,0,0,0,0,0,0,0,0,2000


In [174]:
player_df = player_df.fillna(.000) # the null values are from players that did not attempt the respective shot

In [175]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14855 entries, 0 to 19161
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  14855 non-null  object
 1   Pos     14855 non-null  object
 2   Age     14855 non-null  object
 3   Tm      14855 non-null  object
 4   G       14855 non-null  object
 5   GS      14855 non-null  object
 6   MP      14855 non-null  object
 7   FG      14855 non-null  object
 8   FGA     14855 non-null  object
 9   FG%     14855 non-null  object
 10  3P      14855 non-null  object
 11  3PA     14855 non-null  object
 12  3P%     14855 non-null  object
 13  2P      14855 non-null  object
 14  2PA     14855 non-null  object
 15  2P%     14855 non-null  object
 16  eFG%    14855 non-null  object
 17  FT      14855 non-null  object
 18  FTA     14855 non-null  object
 19  FT%     14855 non-null  object
 20  ORB     14855 non-null  object
 21  DRB     14855 non-null  object
 22  TRB     14855 non-null  obj

In [176]:
# changing columns to the correct Dtype
convert_dict = {'Age': int, 'G': int, 'GS': int,
                'MP': float, 'FG': float, 'FGA': float,
                'FG%': float, '3P': float, '3PA': float,
                '3P%': float, '2P': float, '2PA': float,
                '2P%': float, 'eFG%': float, 'FT': float,
                'FTA': float, 'FT%': float, 'ORB': float,
                'DRB': float, 'TRB': float, 'AST': float,
                'STL': float, 'BLK': float, 'TOV': float,
                'PF': float, 'PTS': float}
player_df = player_df.astype(convert_dict)

In [177]:
player_df.rename(columns={'G': 'Games', 'GS': 'Started', 'Tm': 'Team'}, inplace=True)

In [179]:
player_df['Pos'].value_counts()

Pos
PF          3036
SG          3013
C           3002
PG          2849
SF          2711
SF-SG         36
PG-SG         32
SG-PG         32
SG-SF         30
C-PF          29
PF-C          27
PF-SF         26
SF-PF         24
SG-PF          4
PG-SF          1
SF-C           1
SG-PG-SF       1
SF-PG          1
Name: count, dtype: int64

In [180]:
# standardizing position names to make analysis easier
player_df['Pos'] = player_df['Pos'].str.replace('SF-PG', 'Guard')
player_df['Pos'] = player_df['Pos'].str.replace('SG-PG-SF', 'Guard')
player_df['Pos'] = player_df['Pos'].str.replace('SF-C', 'Forward')
player_df['Pos'] = player_df['Pos'].str.replace('PG-SF', 'Forward')
player_df['Pos'] = player_df['Pos'].str.replace('SG-PF', 'Guard')
player_df['Pos'] = player_df['Pos'].str.replace('SF-PF', 'Forward')
player_df['Pos'] = player_df['Pos'].str.replace('PF-SF', 'Forward')
player_df['Pos'] = player_df['Pos'].str.replace('PF-C', 'Center')
player_df['Pos'] = player_df['Pos'].str.replace('C-PF', 'Center')
player_df['Pos'] = player_df['Pos'].str.replace('SG-SF', 'Guard')
player_df['Pos'] = player_df['Pos'].str.replace('SG-PG', 'Guard')
player_df['Pos'] = player_df['Pos'].str.replace('PG-SG', 'Guard')
player_df['Pos'] = player_df['Pos'].str.replace('SF-SG', 'Forward')
player_df['Pos'] = player_df['Pos'].str.replace('SF', 'Forward')
player_df['Pos'] = player_df['Pos'].str.replace('PG', 'Guard')
player_df['Pos'] = player_df['Pos'].apply(lambda x: re.sub(r'\bC\b', 'Center', x))
player_df['Pos'] = player_df['Pos'].str.replace('SG', 'Guard')
player_df['Pos'] = player_df['Pos'].str.replace('PF', 'Forward')

In [181]:
player_df['Pos'].value_counts()

Pos
Guard      5962
Forward    5835
Center     3058
Name: count, dtype: int64

In [183]:
player_df.to_csv('cleaned_player_stats.csv')

In [46]:
team_df = pd.read_csv('team_ratings.csv')

In [47]:
team_df.head()

,Unnamed: 0,Rk,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,Year
0,0,1,Chicago Bulls,E,C,67,15,0.817,10.44,116.32,105.35,10.97,10.07,116.12,105.52,10.61,1992
1,1,2,Portland Trail Blazers,W,P,57,25,0.695,7.27,111.98,104.71,7.27,6.94,111.85,104.94,6.91,1992
2,2,3,Utah Jazz,W,M,55,27,0.671,6.39,112.68,106.13,6.55,5.70,112.59,106.77,5.82,1992
3,3,4,Phoenix Suns,W,P,53,29,0.646,5.94,112.51,106.70,5.82,5.69,112.50,106.96,5.54,1992
4,4,5,Cleveland Cavaliers,E,C,57,25,0.695,5.45,114.32,108.71,5.61,5.34,114.26,108.73,5.53,1992


In [48]:
team_df.shape

(939, 17)

In [49]:
team_df.drop(columns=['Unnamed: 0', 'Rk'], inplace=True)

In [50]:
team_df.isnull().sum()

Team      0
Conf      0
Div       0
W         0
L         0
W/L%      0
MOV       0
ORtg      0
DRtg      0
NRtg      0
MOV/A     0
ORtg/A    0
DRtg/A    0
NRtg/A    0
Year      0
dtype: int64

In [51]:
team_df['Team'].value_counts()

Team
Chicago Bulls                        32
Los Angeles Lakers                   32
Dallas Mavericks                     32
Denver Nuggets                       32
Minnesota Timberwolves               32
Orlando Magic                        32
Sacramento Kings                     32
Miami Heat                           32
Portland Trail Blazers               32
Milwaukee Bucks                      32
Philadelphia 76ers                   32
Atlanta Hawks                        32
Houston Rockets                      32
Los Angeles Clippers                 32
Golden State Warriors                32
Utah Jazz                            32
Cleveland Cavaliers                  32
New York Knicks                      32
Boston Celtics                       32
Phoenix Suns                         32
San Antonio Spurs                    32
Detroit Pistons                      32
Indiana Pacers                       32
Toronto Raptors                      28
Washington Wizards                 

In [59]:
# we have teams that have relocated or renamed, so we will change their names to the current team name
team_df['Team'] = team_df['Team'].str.replace('Seattle SuperSonics', 'Oklahoma City Thunder')
team_df['Team'] = team_df['Team'].str.replace('Vancouver Grizzlies', 'Memphis Grizzlies')
team_df['Team'] = team_df['Team'].str.replace('Washington Bullets', 'Washington Wizards')
team_df['Team'] = team_df['Team'].str.replace('New Jersey Nets', 'Brooklyn Nets')
team_df['Team'] = team_df['Team'].str.replace('Charlotte Bobcats', 'Charlotte Hornets')
team_df['Team'] = team_df['Team'].str.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans')
team_df['Team'] = team_df['Team'].str.replace('New Orleans Hornets', 'New Orleans Pelicans')

In [182]:
team_df['Team'].value_counts()

Team
Chicago Bulls             32
Atlanta Hawks             32
Dallas Mavericks          32
Denver Nuggets            32
Minnesota Timberwolves    32
Orlando Magic             32
Sacramento Kings          32
Washington Wizards        32
Miami Heat                32
Houston Rockets           32
Milwaukee Bucks           32
Philadelphia 76ers        32
Portland Trail Blazers    32
Brooklyn Nets             32
Los Angeles Lakers        32
Los Angeles Clippers      32
Oklahoma City Thunder     32
Indiana Pacers            32
Detroit Pistons           32
San Antonio Spurs         32
Golden State Warriors     32
Boston Celtics            32
New York Knicks           32
Cleveland Cavaliers       32
Phoenix Suns              32
Utah Jazz                 32
Charlotte Hornets         30
Toronto Raptors           28
Memphis Grizzlies         28
New Orleans Pelicans      21
Name: count, dtype: int64

In [66]:
team_df.to_csv('cleaned_team_ratings.csv')